# <span style="color:darkred">Additive Schwarz  using multiple strategies</span>.

<img align="left" src="Schwarz.jpg" width=120 height=120 />

### [Simon Tavener](https://www.math.colostate.edu/~tavener/) and Andre Leautaud

#### Full text for this notebook can be found at the following [link](https://www.math.colostate.edu/~tavener/andre/DD_multiphysics.pdf)

#### Set path and parameters

In [1]:
import numpy as np
import math
%run DD_Utilities_sjt_al.ipynb
%run AS_Utilities.ipynb
iprint= 6

In [2]:
iparam= -1

In [3]:
if iparam == 5:
    n= 12
    p= 4 
    b= [1,2,5,9]
    e= [3,7,10,12] #List of ends (inclusive) or integer representing overlap
    K= 3
    alpha= 0.5
    if K > 500:
        raise ValueError("K is too large")

In [4]:
if iparam == -1:
    n= 10
    p= 4 
    b= [1,2,5,9]
    e= [3,7,10,10] #List of ends (inclusive) or integer representing overlap
    K= 3
    alpha= 0.5
    if K > 500:
        raise ValueError("K is too large")

In [5]:
if iparam == 0:
    n= 10
    p= 4 
    b= [1,2,4,7]
    e= [3,4,8,10] #List of ends (inclusive) or integer representing overlap
    K= 3
    alpha= 0.5
    if K > 500:
        raise ValueError("K is too large")

In [6]:
# This example shows how to create domains that each overlap by 1 column
#small system
if iparam == 1:
    n= 8
    p= 3 
    b= [1,3,5]
    e= 1
    K= 3
    alpha= 0.5
    if K > 500:
        raise ValueError("K is too large")

In [7]:
# This example shows the optional parameters for set_parameters
if iparam == 2:
    n= 12
    p= -1 #p should be a positive integer or -1
    b= [-1] # an evenly dispersed set of domains will be chosen
    e= [-1] #List of ends or integer representing overlap or [-1] to automaically choose overlap
    K= 6
    alpha= 0.5
    if K > 500:
        raise ValueError("K is too large")

In [8]:
#subdomains contained in others
if iparam == 3:
    n= 10
    p= 5 #p should be a positive integer or -1
    b= [1,1,5,5,7]
    e= [2,4,6,8,10] 
    K= 6
    alpha= 0.5
    if K > 500:
        raise ValueError("K is too large")

In [9]:
#overlap does not connect all subdomains
if iparam == 4:
    n= 9
    p= 4 
    b= [1,3,6,8]
    e= [2,5,7,9]
    K= 3
    alpha= 0.5
    if K > 500:
        raise ValueError("K is too large")

In [10]:
p,b,e= set_parameters(n,p,b,e)  #set_parameters creates a sample set of domains if the user does not fully specify it.
print(p,b,e)
nn,nb,ne,index= fix_domains(n,p,b,e,iprint)

4 [1, 2, 5, 9] [3, 7, 10, 10]


### Establish and solve forward system, $Mx=y$

### Establish subdomains, partitions, number of iterations and random numerical error.  Read and/or save as appropriate

In [11]:
sread= False    #Read system of equations from csv files
swrite= True    #Write system to csv file
mpert= 0.02

M,x,y,xstore= Linear_system(n,sread,iparam)
wpert= Make_wpert(n,p,K,sread,iparam)
if iprint >= 4:
    print(M)
    print('y')
    print(y)
    print('x')
    print(x)
if swrite:
    save_to_csv(n,p,K,alpha,nb,nn,ne,M,y,mpert,wpert,iparam)

[[1.04170220e+01 7.20324493e-01 1.14374817e-04 3.02332573e-01
  1.46755891e-01 9.23385948e-02 1.86260211e-01 3.45560727e-01
  3.96767474e-01 5.38816734e-01]
 [4.19194514e-01 1.06852195e+01 2.04452250e-01 8.78117436e-01
  2.73875932e-02 6.70467510e-01 4.17304802e-01 5.58689828e-01
  1.40386939e-01 1.98101489e-01]
 [8.00744569e-01 9.68261576e-01 1.03134242e+01 6.92322616e-01
  8.76389152e-01 8.94606664e-01 8.50442114e-02 3.90547832e-02
  1.69830420e-01 8.78142503e-01]
 [9.83468338e-02 4.21107625e-01 9.57889530e-01 1.05331653e+01
  6.91877114e-01 3.15515631e-01 6.86500928e-01 8.34625672e-01
  1.82882773e-02 7.50144315e-01]
 [9.88861089e-01 7.48165654e-01 2.80443992e-01 7.89279328e-01
  1.01032260e+01 4.47893526e-01 9.08595503e-01 2.93614148e-01
  2.87775339e-01 1.30028572e-01]
 [1.93669579e-02 6.78835533e-01 2.11628116e-01 2.65546659e-01
  4.91573159e-01 1.00533625e+01 5.74117605e-01 1.46728575e-01
  5.89305537e-01 6.99758360e-01]
 [1.02334429e-01 4.14055988e-01 6.94400158e-01 4.14179270e

### Iterative strategy
$u^{\{k\}}= u^{\{k-1\}} + Br = u^{\{k-1\}} + B(f-Au^{\{k-1\}})$, $\quad k=1, \dots, K,$ where $B$ is an approximation to $A^{-1}$

<ins>Additive<ins>
$A_i= R_i A R_i^\top \in \mathbb{R}^{m_i \times m_i} \\
B_i= R_i^\top A_i^{-1} R_i  = R_i^\top ( R_i A R_i^\top)^{-1} R_i  \in \mathbb{R}^{n \times n} \\
C_i= B_i A   \in \mathbb{R}^{n \times n} \\
f_i= B_i f   \in \mathbb{R}^{n}
$

In [12]:
#Construct domains, R,A,B,C, and f
R= Rmatrices(n,p,nn,index,iprint)
A= Amatrices(M,R,n,p,nn,iprint)
B= Bmatrices(A,R,n,p,nn,iprint)
C= Cmatrices(M,B,n,p,iprint) 
f= fvector(B,y,n,p,iprint)

Print B matrices
i= 0
i= 1
i= 2
i= 3
Print C matrices 
Subdomain 0
[[ 1.00000000e+00  1.77949750e-18  8.01538577e-02  4.07261358e-04
   6.10217977e-02  3.83908799e-02  5.11871986e-02  1.14360700e-02
   1.50244391e-02]
 [ 1.54062210e-18  1.00000000e+00  5.77801722e-02  8.38457850e-02
   8.06524722e-02  3.45956308e-03 -3.31959968e-03  1.24974576e-02
   7.97998113e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.000000

####  $K$ iterations computing subdomain contributions independently [C iteration]
$
u^{\{k\}}= u^{\{k-1\}} + \alpha \sum_{i=1}^p B_i(f-Au^{\{k-1\}}) = u^{\{k-1\}} + \alpha \sum_{i=1}^p ( f_i-C_iu^{\{k-1\}}) 
$

In [13]:
#Perform C iteration
vstore= Citeration_additive(C,f,alpha,n,p,K,iprint)

Citeration_additive: dv on subdomain   0 at iteration   0
[[0.02812641 0.0467045  0.07933329 0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]]
Citeration_additive: dv on subdomain   1 at iteration   0
[[ 0.02812641  0.0467045   0.07933329  0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.04316192  0.06901942  0.01950052  0.08167922  0.05361552
  -0.00852103  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0. 

In [14]:
if iprint >= 4:
    print('vstore')
    print(vstore)

vstore
[[ 0.01406321]
 [ 0.04493321]
 [ 0.07417635]
 [ 0.00975026]
 [ 0.08299801]
 [ 0.05119085]
 [-0.01127987]
 [ 0.03878446]
 [ 0.05318585]
 [ 0.09025737]
 [ 0.01651008]
 [ 0.03960102]
 [ 0.06055062]
 [ 0.01074402]
 [ 0.0783464 ]
 [ 0.04522442]
 [-0.01928282]
 [ 0.05732939]
 [ 0.04012172]
 [ 0.08013145]
 [ 0.01805479]
 [ 0.03919551]
 [ 0.06191733]
 [ 0.01080999]
 [ 0.07836086]
 [ 0.04605434]
 [-0.01846651]
 [ 0.06646885]
 [ 0.04070673]
 [ 0.08105859]]


####  $K$ iterations combining $p$  (parallel) subdomain solves [D iteration]

<ins>Additive Schwarz<ins>
    
$u^{\{k\}}= D u^{\{k-1\}} + g, \qquad 
D = I-\alpha \sum_{i=1}^p C_i, \quad g= \alpha \sum_{i=1}^p f_i$


In [15]:
# Perform D iteration
#Construct D and g and solve
D= Dmatrix_additive(C,alpha,n,p,iprint)

dmatrix_additive:  matrix
[[ 5.00e-01 -0.00e+00 -0.00e+00 -1.17e-02 -7.00e-03 -2.30e-03 -7.60e-03
  -1.48e-02 -1.86e-02 -2.53e-02]
 [-1.92e-02  0.00e+00  0.00e+00 -4.01e-02 -2.00e-04 -3.05e-02 -1.92e-02
  -4.77e-02 -9.20e-03 -1.08e-02]
 [-3.35e-02 -0.00e+00  0.00e+00 -2.89e-02 -4.19e-02 -4.03e-02 -1.70e-03
   5.50e-03 -1.10e-02 -7.73e-02]
 [ 2.20e-03  0.00e+00  0.00e+00  5.00e-01 -0.00e+00 -0.00e+00  0.00e+00
  -3.70e-02  3.90e-03 -2.97e-02]
 [-9.37e-02 -3.34e-02 -9.90e-03 -3.58e-02  0.00e+00  0.00e+00 -0.00e+00
  -8.00e-03 -9.20e-03  6.00e-04]
 [ 7.50e-03 -2.71e-02 -5.30e-03 -8.00e-03 -0.00e+00  0.00e+00 -0.00e+00
  -3.30e-03 -2.62e-02 -3.18e-02]
 [-1.40e-03 -1.36e-02 -2.84e-02 -1.51e-02  0.00e+00  0.00e+00  0.00e+00
  -2.19e-02 -4.26e-02 -2.22e-02]
 [-3.92e-02  9.00e-04 -1.00e-04 -3.39e-02 -0.00e+00 -0.00e+00 -0.00e+00
   5.00e-01 -0.00e+00 -0.00e+00]
 [-7.81e-02 -4.99e-02 -6.63e-02 -2.46e-02 -1.15e-02 -4.14e-02 -1.76e-02
  -4.37e-02  0.00e+00 -0.00e+00]
 [-7.00e-03 -8.64e-02 -3.94e-

In [16]:
g= gvector_additive(f,alpha,n,p,iprint)

Print g vector
[[ 0.01406321]
 [ 0.04493321]
 [ 0.07417635]
 [ 0.00975026]
 [ 0.08299801]
 [ 0.05119085]
 [-0.01127987]
 [ 0.03878446]
 [ 0.05318585]
 [ 0.09025737]]


In [17]:
# print('Perform D iteration \n')
wstore= Diteration_additive(D,g,n,p,K,iprint)

Diteration_additive: w at iteration  0
[[ 0.03254028]
 [-0.05898534]
 [-0.08616912]
 [ 0.350852  ]
 [-0.00116961]
 [-0.02697311]
 [-0.11543052]
 [ 0.23273595]
 [-0.13940163]
 [-0.08604378]]
Diteration_additive: w at iteration  1
[[ 0.02849585]
 [ 0.02439517]
 [ 0.07374539]
 [ 0.178648  ]
 [ 0.06958568]
 [ 0.05632101]
 [-0.01062287]
 [ 0.14195257]
 [ 0.04367462]
 [ 0.08000111]]
Diteration_additive: w at iteration  2
[[ 0.02072962]
 [ 0.02766704]
 [ 0.05700395]
 [ 0.09167261]
 [ 0.07089577]
 [ 0.04478639]
 [-0.02318957]
 [ 0.10261056]
 [ 0.03132378]
 [ 0.07079752]]
norm(D)=  9.076747e-01
Asymptotic solution: z= inv(I-D)*g
K=     3, norm(w-z)=  8.868375e-02


In [18]:
if iprint >= 2:
    print('wstore')
    print(wstore)

wstore
[[ 0.03254028]
 [-0.05898534]
 [-0.08616912]
 [ 0.350852  ]
 [-0.00116961]
 [-0.02697311]
 [-0.11543052]
 [ 0.23273595]
 [-0.13940163]
 [-0.08604378]
 [ 0.02849585]
 [ 0.02439517]
 [ 0.07374539]
 [ 0.178648  ]
 [ 0.06958568]
 [ 0.05632101]
 [-0.01062287]
 [ 0.14195257]
 [ 0.04367462]
 [ 0.08000111]
 [ 0.02072962]
 [ 0.02766704]
 [ 0.05700395]
 [ 0.09167261]
 [ 0.07089577]
 [ 0.04478639]
 [-0.02318957]
 [ 0.10261056]
 [ 0.03132378]
 [ 0.07079752]]


#### Perform $K$ iterations as single matrix solve [U iteration]

$K=6$ iterations of multiplicative Schwarz can be written as the $nK$-dimensional system of equations

<ins>Additive Schwarz<ins>

$Uz=
\pmatrix{
 I & 0 & 0 & 0 & 0 & 0   \cr
 -D &I & 0 & 0 & 0 & 0   \cr
 0 & -D & I & 0 & 0 & 0   \cr
 0 & 0 & -D & I & 0 & 0   \cr
 0 & 0 & 0 & -D & I & 0   \cr
 0 & 0 & 0 & 0 & -D & I
}$
$\pmatrix{
v^{\{1\}}  \cr
v^{\{2\}}  \cr
v^{\{3\}}  \cr
v^{\{4\}}  \cr
v^{\{5\}}  \cr
v^{\{6\}}
}$
$\pmatrix{
g  \cr
g  \cr
g  \cr
g  \cr
g  \cr
g
}
=h$

In [19]:
# Perform U iteration
# Construct U and h
U= Umatrix_additive(D,n,K,iprint)
h= hvector_additive(g,n,K,iprint)

U matrix 

[[ 1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0

In [20]:
# Solve using Uz= h
zstore= Uhsolve_additive(U,h,n,K,iprint)

Uhsolve_additive: z at iteration  0
[ 0.01406321  0.04493321  0.07417635  0.00975026  0.08299801  0.05119085
 -0.01127987  0.03878446  0.05318585  0.09025737]
Uhsolve_additive: z at iteration  1
[ 0.01651008  0.03960102  0.06055062  0.01074402  0.0783464   0.04522442
 -0.01928282  0.05732939  0.04012172  0.08013145]
Uhsolve_additive: z at iteration  2
[ 0.01805479  0.03919551  0.06191733  0.01080999  0.07836086  0.04605434
 -0.01846651  0.06646885  0.04070673  0.08105859]


In [21]:
if iprint >= 4:
    print('zstore')
    print(zstore)

zstore
[[ 0.01406321]
 [ 0.04493321]
 [ 0.07417635]
 [ 0.00975026]
 [ 0.08299801]
 [ 0.05119085]
 [-0.01127987]
 [ 0.03878446]
 [ 0.05318585]
 [ 0.09025737]
 [ 0.01651008]
 [ 0.03960102]
 [ 0.06055062]
 [ 0.01074402]
 [ 0.0783464 ]
 [ 0.04522442]
 [-0.01928282]
 [ 0.05732939]
 [ 0.04012172]
 [ 0.08013145]
 [ 0.01805479]
 [ 0.03919551]
 [ 0.06191733]
 [ 0.01080999]
 [ 0.07836086]
 [ 0.04605434]
 [-0.01846651]
 [ 0.06646885]
 [ 0.04070673]
 [ 0.08105859]]


#### Compare Solutions from all three methods

In [22]:
solution_compare(xstore,vstore,wstore,zstore,iprint)

norm(exact - iterative) solutions    =  9.293539e-03
Difference between vstore and wstore =  5.722791e-01
Difference between wstore and zstore =  5.722791e-01
Difference between vstore and zstore =  3.736709e-17
[ 1.01968621e-03 -9.00233347e-04 -7.67461633e-06 -1.30171945e-03
 -3.06714085e-04 -6.85729980e-06 -3.96111796e-04  9.01967849e-03
 -1.02579521e-03 -4.08984022e-04]
[[-1.84770708e-02]
 [ 1.03918545e-01]
 [ 1.60345476e-01]
 [-3.41101740e-01]
 [ 8.41676200e-02]
 [ 7.81639569e-02]
 [ 1.04150653e-01]
 [-1.93951486e-01]
 [ 1.92587482e-01]
 [ 1.76301158e-01]
 [-1.19857729e-02]
 [ 1.52058435e-02]
 [-1.31947765e-02]
 [-1.67903984e-01]
 [ 8.76071665e-03]
 [-1.10965906e-02]
 [-8.65994633e-03]
 [-8.46231835e-02]
 [-3.55289797e-03]
 [ 1.30344738e-04]
 [-2.67482413e-03]
 [ 1.15284736e-02]
 [ 4.91338205e-03]
 [-8.08626138e-02]
 [ 7.46509267e-03]
 [ 1.26795430e-03]
 [ 4.72305593e-03]
 [-3.61417128e-02]
 [ 9.38294946e-03]
 [ 1.02610688e-02]]
[[ 1.84770708e-02]
 [-1.03918545e-01]
 [-1.60345476e-

### Add discretization error and compute error estimate

#### C iteration with error

In [23]:
psi= np.ones((n,1))

# Solve system with "numerical error" for every solve
vglobal_store,vaglobal_store,rglobal_store= Citeration_additive_approx(C,f,alpha,n,p,K,mpert,wpert,iprint)

if iprint >= 4:
    print('vglobal_store')
    print(vglobal_store)
    print('vaglobal_store')
    print(vaglobal_store)
    print('rglobal_store')
    print(rglobal_store)    

Citeration_additive_approx: Outer iteration  0
Difference = 3.9121e-02
dmatrix_additive:  matrix
[[ 5.00e-01 -0.00e+00 -0.00e+00 -1.17e-02 -7.00e-03 -2.30e-03 -7.60e-03
  -1.48e-02 -1.86e-02 -2.53e-02]
 [-1.92e-02  0.00e+00  0.00e+00 -4.01e-02 -2.00e-04 -3.05e-02 -1.92e-02
  -4.77e-02 -9.20e-03 -1.08e-02]
 [-3.35e-02 -0.00e+00  0.00e+00 -2.89e-02 -4.19e-02 -4.03e-02 -1.70e-03
   5.50e-03 -1.10e-02 -7.73e-02]
 [ 2.20e-03  0.00e+00  0.00e+00  5.00e-01 -0.00e+00 -0.00e+00  0.00e+00
  -3.70e-02  3.90e-03 -2.97e-02]
 [-9.37e-02 -3.34e-02 -9.90e-03 -3.58e-02  0.00e+00  0.00e+00 -0.00e+00
  -8.00e-03 -9.20e-03  6.00e-04]
 [ 7.50e-03 -2.71e-02 -5.30e-03 -8.00e-03 -0.00e+00  0.00e+00 -0.00e+00
  -3.30e-03 -2.62e-02 -3.18e-02]
 [-1.40e-03 -1.36e-02 -2.84e-02 -1.51e-02  0.00e+00  0.00e+00  0.00e+00
  -2.19e-02 -4.26e-02 -2.22e-02]
 [-3.92e-02  9.00e-04 -1.00e-04 -3.39e-02 -0.00e+00 -0.00e+00 -0.00e+00
   5.00e-01 -0.00e+00 -0.00e+00]
 [-7.81e-02 -4.99e-02 -6.63e-02 -2.46e-02 -1.15e-02 -4.14e-02 -

#### Calculate the discretization error and effectivity ratio

In [24]:
vglobal_store

array([[ 0.01406321],
       [ 0.04493321],
       [ 0.07417635],
       [ 0.00975026],
       [ 0.08299801],
       [ 0.05119085],
       [-0.01127987],
       [ 0.03878446],
       [ 0.05318585],
       [ 0.09025737],
       [ 0.01651008],
       [ 0.03960102],
       [ 0.06055062],
       [ 0.01074402],
       [ 0.0783464 ],
       [ 0.04522442],
       [-0.01928282],
       [ 0.05732939],
       [ 0.04012172],
       [ 0.08013145],
       [ 0.01805479],
       [ 0.03919551],
       [ 0.06191733],
       [ 0.01080999],
       [ 0.07836086],
       [ 0.04605434],
       [-0.01846651],
       [ 0.06646885],
       [ 0.04070673],
       [ 0.08105859]])

In [25]:
# Adjoint solve
phiglobal_store= Citeration_additive_adjoint(C,psi,alpha,n,p,K,iprint)
if iprint >= 4:
    print('phiglobal_store')
    print(phiglobal_store.T)

# Error estimate= (r,\phi)
qoi_discretization_error_estimate= (rglobal_store.T @ phiglobal_store)[0,0]

if iprint >= 4:
    print(f'norm(r_store) = {np.linalg.norm(rglobal_store):13.6e}, norm(phi_store)= {np.linalg.norm(phiglobal_store):13.6e}')

# Compute error and error in QoI directly
vsoln= vglobal_store[n * (K-1): n * K, 0]
vasoln= vaglobal_store[n * K: n * (K+1), 0]
verror= vsoln.T-vasoln
qoi_discretization_error= (psi.T @ verror.T)[0]

# Calculate effectivity ratio
effectivity_ratio_disc= qoi_discretization_error_estimate/ qoi_discretization_error
print(f'QoI_discretization_error          = {qoi_discretization_error:13.6e}')
print(f'QoI_discretization_error_estimate = {qoi_discretization_error_estimate:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_disc:13.3f}')

10 4 3
Citeration_additive_adjoint: k= 2, i= 3
[0.09299157 0.1426893  0.11031026 0.08401244 0.06150763 0.10501703
 0.12026289 0.14134196 1.         1.        ]
Citeration_additive_adjoint: k= 2, i= 2
[0.24011473 0.2762992  0.18845559 0.25017223 1.         1.
 1.         1.         1.         1.        ]
Citeration_additive_adjoint: k= 2, i= 1
[0.19139057 1.         1.         1.         1.         1.
 1.         0.17693531 0.16470126 0.24749743]
Citeration_additive_adjoint: k= 2, i= 0
[1.         1.         1.         0.16141379 0.09831205 0.146318
 0.05707609 0.0775008  0.06123098 0.14550909]
Citeration_additive_adjoint: k= 1, i= 3
[-0.01140588 -0.02358856 -0.01599976 -0.01404074 -0.01015903 -0.01372646
 -0.02069275 -0.02047867 -0.11296612 -0.19650326]
Citeration_additive_adjoint: k= 1, i= 2
[ 0.00830405 -0.03675025 -0.02235846 -0.00102241 -0.07990984 -0.12566751
 -0.08866949  0.30211096 -0.11296612 -0.19650326]
Citeration_additive_adjoint: k= 1, i= 1
[-0.02612531 -0.20949425 -0.14938

##### Calculate the total error and effectivity ratio

In [26]:
# Adjoint solve
phi= np.linalg.solve(M.T,psi)

# Residual
r= y.T - M @ vasoln

# Error estimate= (r,\phi)
qoi_total_error_estimate= (r @ phi)[0,0]

# Compute error in QoI directly
total_error= xstore.reshape(-1) - vasoln.T
qoi_total_error= psi.reshape(-1)@total_error

# Calculate effectivity ratio
effectivity_ratio_total= qoi_total_error_estimate / qoi_total_error
print(f'QoI_total_error                   = {qoi_total_error:13.6e}')
print(f'QoI_total_error_estimate          = {qoi_total_error_estimate:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_total:13.6e}')

QoI_total_error                   = -1.185734e-01
QoI_total_error_estimate          = -1.185734e-01
Effectivity ratio                 =  1.000000e+00


#### Calculate iteration error as the difference between the total error and the discretization error

In [27]:
qoi_iteration_error= qoi_total_error-qoi_discretization_error
print(f'QoI_iteration_error               = {qoi_iteration_error:13.6e}')

QoI_iteration_error               =  5.685275e-03


#### D iteration with error

In [28]:
# Diteration with error as a check on the accumulation of error
print('Perform D iteration with error')

Perform D iteration with error


In [29]:
w_store,wa_store,wr_store= Diteration_additive_approx(D,g,alpha,n,p,K,wpert,mpert,iprint);
print(f'norm(w_store-vglobal_store)= {np.linalg.norm(w_store-vglobal_store):13.6e}')

norm(w_store-vglobal_store)=  2.194271e-17


In [30]:
print(f'norm(wa_store-vaglobal_store)= {np.linalg.norm(wa_store-vaglobal_store):13.6e}')

norm(wa_store-vaglobal_store)=  3.707401e-17


In [31]:
print(f'norm(wr_store-rglobal_store)= {np.linalg.norm(wr_store-rglobal_store):13.6e}')

norm(wr_store-rglobal_store)=  3.778752e-17


### U iteration with error

In [32]:
print('Solving U system with error')

Solving U system with error


In [33]:
zexact,zapprox,zresid= Uhsolve_additive_approx(U,h,n,p,K,alpha,wpert,mpert,iprint)

In [34]:
if iprint >= 4:
    print('zexact')
    print(zexact.T)
    print('zapprox')
    print(zapprox.T)
    print('zresid')
    print(zresid.T)

zexact
[[ 0.01406321  0.04493321  0.07417635  0.00975026  0.08299801  0.05119085
  -0.01127987  0.03878446  0.05318585  0.09025737  0.01651008  0.03960102
   0.06055062  0.01074402  0.0783464   0.04522442 -0.01928282  0.05732939
   0.04012172  0.08013145  0.01805479  0.03919551  0.06191733  0.01080999
   0.07836086  0.04605434 -0.01846651  0.06646885  0.04070673  0.08105859]]
zapprox
[[ 0.00562664  0.06017386  0.08478564  0.0048041   0.08177229  0.04172533
  -0.01440642  0.01730134  0.05675605  0.11294769  0.00948784  0.00044436
   0.0800647   0.06529433  0.07740295  0.05172621  0.00161321  0.05425935
   0.03864669  0.0848359   0.00529248  0.04507052  0.11376702  0.02940006
   0.0892357   0.01959617  0.01636221  0.0723413   0.0803306   0.06861025]]
zresid
[[ 0.00843656 -0.01524065 -0.01060929  0.00494616  0.00122572  0.00946552
   0.00312655  0.02148312 -0.0035702  -0.02269031  0.00259312  0.04061311
  -0.02056065 -0.05690719  0.00144799 -0.00773893 -0.02150275 -0.00716088
   0.0021907

In [35]:
if iprint >= 2:
    print(f'Norm of difference between zexact   and vglobal_store  {np.linalg.norm(zexact-vglobal_store):13.6e}')
    print(f'Norm of difference between zapprox  and vaglobal_store {np.linalg.norm(zapprox-vaglobal_store[n:]):13.6e}')
    print(f'Norm of difference between zresid   and rglobal_store  {np.linalg.norm(zresid-rglobal_store):13.6e}')

Norm of difference between zexact   and vglobal_store   3.736709e-17
Norm of difference between zapprox  and vaglobal_store  3.077902e-02
Norm of difference between zresid   and rglobal_store   3.031027e-02


#### Calculate the discretization error and effectivity ratio

In [36]:
# Adjoint data (Adjoint system is of size nK)
psi_nK= np.zeros((n*K,1))
psi_nK[n * (K-1):n * K,0:1]= psi

# Solve adjoint equation
phi_nK= np.linalg.solve(U.T,psi_nK)
if iprint >= 2:
    print(f'Norm of difference between phi_nK and phiglobal_store {np.linalg.norm(phi_nK-phiglobal_store):13.6e}')

Norm of difference between phi_nK and phiglobal_store  3.264197e-16


In [37]:
# Compute error estimate
qoi_discretization_error_estimate= (phi_nK.T @ zresid)[0,0]

# Compute error and error in QoI directly
zsoln= zexact[-10:,0:1]
zasoln= zapprox[-10:,0:1]
zerror= zsoln - zasoln
qoi_discretization_error= (psi.T @ zerror)[0,0]

#### Calculate the effectivity ratio

In [38]:
effectivity_ratio_disc= qoi_discretization_error_estimate / qoi_discretization_error
print(f'QoI_discretization_error          = {qoi_discretization_error:13.6e}' )
print(f'QoI_discretization_error_estimate = {qoi_discretization_error_estimate:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_disc:13.3f}')

QoI_discretization_error          = -1.158458e-01
QoI_discretization_error_estimate = -1.158458e-01
Effectivity ratio                 =         1.000


#### Calculate the total error and effectivity ratio

In [39]:
# Residual
r= y - M @ zasoln

# Error estimate= (r,\phi)
qoi_total_error_estimate= r.T @ phi

# Compute error in QoI directly
total_error= xstore - zasoln
qoi_total_error= psi.T @ total_error

#Effectivity ratio
effectivity_ratio_total= qoi_total_error_estimate / qoi_total_error
print(f'QoI_total_error                   = {qoi_total_error[0,0]:13.6e}')
print(f'QoI_total_error_estimate          = {qoi_total_error_estimate[0,0]:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_total[0,0]:13.3f}')

QoI_total_error                   = -1.101605e-01
QoI_total_error_estimate          = -1.101605e-01
Effectivity ratio                 =         1.000


#### Calculate the iteration error as the difference between the total error and the discretization error

In [40]:
qoi_iteration_error= qoi_total_error - qoi_discretization_error
print(f'QoI_iteration_error               = {qoi_iteration_error[0,0]:13.6e}')

QoI_iteration_error               =  5.685275e-03
